#Librerías

In [ ]:
import streamlit as st
import pandas as pd
import arff
from ludwig.api import LudwigModel
import os
import tempfile
import json
import plotly.express as px

#Diseño y estilos CSS personalizados

In [ ]:
st.set_page_config(page_title="Entrenador Ludwig", layout="centered")

st.markdown("""
    <style>
    /* Fondo general */
    body {
        background-color: #e6efff;
    }

    .stApp {
        background-color: #f0f4f8;
        font-family: 'Segoe UI', sans-serif;
        padding: 2rem;
    }

    /* Botones */
    .stButton>button, .stDownloadButton>button {
        background-color: #4CAF50;
        color: white;
        border-radius: 8px;
        transition: 0.3s ease;
    }

    .stButton>button:hover, .stDownloadButton>button:hover {
        background-color: #45a049;
        color: black;
        transform: scale(1.02);
    }

    /* File uploader */
    .stFileUploader {
        border: 1px solid #d1d1d1;
        border-radius: 6px;
        padding: 10px;
        background-color: #ffffff;
    }

    /* Ajuste de titulos */
    h1, h2, h3 {
        color: #2c3e50;
    }

    </style>
""", unsafe_allow_html=True)

#Inicio y cargado del YML y del dataset

In [ ]:
st.title("🎓 Entrenador de modelos con Ludwig")
st.markdown("Carga un dataset `.arff` y un archivo `.yml` para entrenar tu modelo. Luego podrás visualizar y descargar las métricas y predicciones.")

arff_file = st.file_uploader("📂 Sube tu archivo `.arff`", type=["arff"])
yaml_file = st.file_uploader("⚙️ Sube tu archivo `.yml` de configuración", type=["yml", "yaml"])


#Entrenamiento y muestra de las métricas

In [ ]:
if arff_file and yaml_file:
    st.success("¡Archivos cargados correctamente!")

    if st.button("🚀 Entrenar modelo"):
        with st.spinner("Entrenando el modelo..."):

            # Guardar archivos temporalmente
            with tempfile.NamedTemporaryFile(delete=False, suffix=".arff") as temp_arff:
                temp_arff.write(arff_file.read())
                temp_arff_path = temp_arff.name

            with tempfile.NamedTemporaryFile(delete=False, suffix=".yml") as temp_yaml:
                temp_yaml.write(yaml_file.read())
                temp_yaml_path = temp_yaml.name

            try:
                # Leer .arff
                with open(temp_arff_path) as f:
                    arff_data = arff.load(f)
                    df = pd.DataFrame(arff_data['data'], columns=[attr[0] for attr in arff_data['attributes']])

                # Entrenar y evaluar
                model = LudwigModel(config=str(temp_yaml_path))
                results = model.train(dataset=df)
                eval_stats, _, _ = model.evaluate(dataset=df)

                # === Mostrar métricas ===
                st.subheader("📊 Métricas de Evaluación")
                st.json(eval_stats)

                # === Mostrar gráfica interactiva con Plotly ===
                if "class" in eval_stats:
                    metrics = eval_stats["class"]
                    st.subheader("📈 Visualización de métricas")

                    metrics_df = pd.DataFrame({
                        "Métrica": list(metrics.keys()),
                        "Valor": list(metrics.values())
                    })

                    fig = px.bar(metrics_df, x="Métrica", y="Valor", title="Métricas del modelo", color="Métrica", text_auto=".2f")
                    fig.update_layout(yaxis=dict(range=[0, 1]), showlegend=False)
                    st.plotly_chart(fig, use_container_width=True)

                # === Mostrar predicciones ===
                st.subheader("🔮 Predicciones")
                predictions, _ = model.predict(dataset=df)
                st.dataframe(predictions.head())

                # === Botón para descargar predicciones ===
                pred_csv = predictions.to_csv(index=False).encode("utf-8")
                st.download_button(
                    label="⬇️ Descargar predicciones",
                    data=pred_csv,
                    file_name="predicciones.csv",
                    mime="text/csv"
                )

                # === Botón para descargar métricas ===
                metrics_json = json.dumps(eval_stats, indent=2)
                st.download_button(
                    label="⬇️ Descargar métricas",
                    data=metrics_json,
                    file_name="metricas.json",
                    mime="application/json"
                )

            except Exception as e:
                st.error(f"❌ Error durante el entrenamiento: {e}")

            finally:
                # Limpiar archivos temporales
                os.remove(temp_arff_path)
                os.remove(temp_yaml_path)

else:
    st.info("Sube ambos archivos para activar el entrenamiento.")